In [ ]:
%matplotlib inline
from jupyterthemes import jtplot
jtplot.style(theme="onedork", context="notebook", ticks=True, grid=True)

import numpy as np
import h5py
import os
import sys
import glob

import matplotlib.pyplot as plt
import scipy.stats
import seaborn as sns
import pandas as pd
import tikzplotlib

# from tqdm import tqdm
from tqdm.notebook import tqdm_notebook as tqdm

import helper.circular
import fastpli.io
import fastpli.tools
import fastpli.objects
import fastpli.analysis

sim_path = "output/simulation/*.h5"
out_file = "output/analysis/"

In [ ]:
df_pm = pd.read_pickle(os.path.join(out_file, "pm.pkl"))
df_lap = pd.read_pickle(os.path.join(out_file, "lap.pkl"))

In [ ]:
data = df_pm[(df_pm.f0 == 0.0) & (df_pm.f1 == 0.0)].explode("epa_dir")
grid = sns.FacetGrid(data,
                     col="omega",
                     hue="omega",
                     palette=sns.color_palette("dark", 10),
                     col_wrap=5,
                     height=2.5)
grid.map(plt.scatter, "psi", "epa_dir", marker="o")
grid.fig.tight_layout(w_pad=1)

In [ ]:
data = df_pm[(df_pm.f0 == 0.0) & (df_pm.f1 == 0.0)].explode("epa_dir")

fig, axes = plt.subplots(2, 5)
for i, omega in enumerate(data.omega.unique()):
    sns.boxplot(x="psi",
                y="epa_dir",
                data=data[data.omega == omega],
                orient='v',
                color=sns.color_palette("dark", 10)[i],
                ax=axes[i // 5, i % 5])

data = df_pm[(df_pm.f0 == 0.0) & (df_pm.f1 == 0.0)].explode("epa_ret")
fig, axes = plt.subplots(2, 5)
for i, omega in enumerate(data.omega.unique()):
    sns.boxplot(x="psi",
                y="epa_ret",
                data=data[data.omega == omega],
                orient='v',
                color=sns.color_palette("dark", 10)[i],
                ax=axes[i // 5, i % 5])

In [ ]:
skip = 1
for f0_inc in tqdm(df_pm.f0.unique()):
    for f1_rot in tqdm(df_pm[df_pm.f0 == f0_inc].f1.unique(), leave=False):
        hist_bin = lambda n_phi: np.linspace(
            0, np.pi, n_phi + 1, endpoint=True)

        epa_dir = df_pm[(df_pm.f0 == f0_inc)
                        & (df_pm.f1 == f1_rot)].explode("epa_dir")
        rofl_dir = df_pm[(df_pm.f0 == f0_inc)
                         & (df_pm.f1 == f1_rot)].explode("rofl_dir")

        fig, axs = plt.subplots(int(np.ceil(len(epa_dir.psi.unique()) / skip)),
                                int(np.ceil(
                                    len(epa_dir.omega.unique()) / skip)),
                                subplot_kw={'projection': 'polar'},
                                figsize=(30, 30))
        axs = np.atleast_2d(axs)

        for j, _ in enumerate(epa_dir.omega.unique()[::skip]):
            for i, _ in enumerate(epa_dir.psi.unique()[::skip]):
                axs[i, j].set_xticklabels([])
                axs[i, j].set_yticklabels([])

        for j, omega in enumerate(
                tqdm(epa_dir.omega.unique()[::skip], leave=False)):
            for i, psi in enumerate(
                    tqdm(epa_dir[epa_dir.omega == omega].psi.unique()[::skip],
                         leave=False)):

                if os.path.isfile(
                        f"../data/models/cube_2pop_psi_{psi:.1f}_omega_{omega:.1f}_.solved.pkl"
                ):
                    continue

                data = epa_dir[(epa_dir.omega == omega)
                               & (epa_dir.psi == psi)].epa_dir.to_numpy(
                                   dtype=float)
                data = helper.circular.remap(np.deg2rad(data), np.pi, 0)
                h, x = np.histogram(data, hist_bin(72), density=True)
                x = x[:-1] + x[1] - x[0]
                x = np.append(np.concatenate((x, x + np.pi), axis=0), x[0])
                h = np.append(np.concatenate((h, h), axis=0), h[0])
                axs[i, j].plot(x, h)
                h_max = np.max(h)

                # rofl
                data = rofl_dir[(epa_dir.omega == omega)
                                & (epa_dir.psi == psi)].rofl_dir.to_numpy(
                                    dtype=float)
                data = helper.circular.remap(np.deg2rad(data), np.pi, 0)
                h, x = np.histogram(data, hist_bin(72), density=True)
                x = x[:-1] + x[1] - x[0]
                x = np.append(np.concatenate((x, x + np.pi), axis=0), x[0])
                h = np.append(np.concatenate((h, h), axis=0), h[0])
                axs[i, j].plot(x, h)
                h_max = np.max(h)

                # ground truth
                df_o = pd.read_pickle(
                    f"../data/models/cube_2pop_psi_{psi:.1f}_omega_{omega:.1f}_.solved.pkl"
                )
                phi = df_o.phi.to_numpy(float)

                phi = helper.circular.remap(phi, np.pi, 0)

                h, x = np.histogram(phi, hist_bin(180), density=True)
                x = x[:-1] + x[1] - x[0]
                x = np.append(np.concatenate((x, x + np.pi), axis=0), x[0])
                h = np.append(np.concatenate((h, h), axis=0), h[0])
                h = h / np.max(h) * h_max
                axs[i, j].plot(x, h)

        fig.savefig(f"phi_f0_{f0_inc}_f1_{f1_rot}.svg")

In [ ]:
import helper.spherical_harmonics
import helper.schilling

skip = 1
df_acc = pd.DataFrame()
for f0_inc in tqdm(df_pm.f0.unique()):
    for f1_rot in tqdm(df_pm[df_pm.f0 == f0_inc].f1.unique(), leave=False):
        hist_bin = lambda n_phi: np.linspace(
            0, np.pi, n_phi + 1, endpoint=True)

        rofl_dir = df_pm[(df_pm.f0 == f0_inc)
                         & (df_pm.f1 == f1_rot)].explode("rofl_dir")
        rofl_inc = df_pm[(df_pm.f0 == f0_inc)
                         & (df_pm.f1 == f1_rot)].explode("rofl_inc")

        for  psi in rofl_dir.psi.unique()[::skip]:
            for omega in rofl_dir[rofl_dir.psi == psi].omega.unique()[::skip]:
                if not os.path.isfile(
                        f"../data/models/cube_2pop_psi_{psi:.1f}_omega_{omega:.1f}_.solved.pkl"
                ):
                    continue

                # rofl
                phi = rofl_dir[(rofl_dir.omega == omega)
                               & (rofl_dir.psi == psi)].rofl_dir.to_numpy(
                                   dtype=float)
                theta = np.pi / 2 - rofl_inc[
                    (rofl_inc.omega == omega)
                    & (rofl_inc.psi == psi)].rofl_inc.to_numpy(dtype=float)
                sh0 = helper.spherical_harmonics.real_spherical_harmonics(
                    phi, theta, 6)

                # ground truth
                df_o = pd.read_pickle(
                    f"../data/models/cube_2pop_psi_{psi:.1f}_omega_{omega:.1f}_.solved.pkl"
                )
                phi = df_o[(df_o.f0_inc == f0_inc)
                           & (df_o.f1_rot == f1_rot)].phi.explode().to_numpy(float)
                theta = df_o[(df_o.f0_inc == f0_inc)
                             & (df_o.f1_rot == f1_rot)].theta.explode().to_numpy(float)
                sh1 = helper.spherical_harmonics.real_spherical_harmonics(
                    phi, theta, 6)

                df_acc = df_acc.append(
                    {
                        'f0_inc':
                        f0_inc,
                        'f1_rot':
                        f1_rot,
                        'omega':
                        omega,
                        'psi':
                        psi,
                        'acc':
                        np.sum(np.multiply(sh0, sh1)) /
                        (np.sqrt(np.sum(np.multiply(sh0, sh0))) *
                         np.sqrt(np.sum(np.multiply(sh1, sh1))))
                    },
                    ignore_index=True)

In [ ]:
import helper.spherical_interpolation

omega=0.5
psi = 0.3
v = np.array([1,0,0])

f0_inc = df_acc[(df_acc.omega==omega) & (df_acc.psi==psi)].f0_inc.to_numpy(float)
f1_rot = df_acc[(df_acc.omega==omega) & (df_acc.psi==psi)].f1_rot.to_numpy(float)
data = df_acc[(df_acc.omega==omega) & (df_acc.psi==psi)].acc.to_numpy(float)

phi = []
theta = []
for f0, f1 zip(f0_inc, f1_rot):
    rot_inc = fastpli.tools.rotation.y(-np.deg2rad(f0))
    rot_phi = fastpli.tools.rotation.x(np.deg2rad(f1))
    rot = np.dot(rot_inc, rot_phi)
    v_ = np.dot(rot, v)
    theta.extend(np.arccos(v_[2]))
    phi.extend(np.arctan2(v_[1], v_[0]))

# phi = [0, 0]
# theta = [0, np.pi / 2]
# data = [20, 0]
x, y, z, data_i = helper.spherical_interpolation.on_mesh(phi, theta, data, 40, 40)

import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(x, y, z, facecolors=plt.cm.jet(data_i / data_i.max()))
ax.set_xlabel('$X$')
ax.set_ylabel('$Y$')
plt.show()